# Projet préparez des données pour un organisme de santé publique

In [ ]:
# To run the notebook
%pip install jupyter

# To draw plots
%pip install matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
# To draw plots in the notebook
%matplotlib inline

# To manipulate dataFrames
%pip install pandas
import pandas as pd

# To use quick functions (mainly on arrays)
%pip install numpy
import numpy as np

# To plot prettiers graphs simpler
%pip install seaborn
import seaborn as sns
sns.set()


# Allow to omit the warnings
%pip install warnings
import warnings
warnings.filterwarnings(action='ignore')

# To print dataframes in a nice way
%pip install dataframe_image
import dataframe_image as dfi

# To be able to encode dataframe to display them as pictures
%pip install unicodedata
import unicodedata

# To use widgets to interact with the notebook
%pip install ipywidgets
import ipywidgets as widgets

# To use data science models
%pip install sklearn
from sklearn import linear_model
from sklearn.impute import SimpleImputer

# To make reports
%pip install scipy
from scipy.cluster.hierarchy import dendrogram



from IPython.display import HTML

In [ ]:
url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip'

# Download file from url
import urllib.request
urllib.request.urlretrieve(url, 'fr.openfoodfacts.org.products.csv.zip')

# Unzip file
import zipfile
with zipfile.ZipFile('fr.openfoodfacts.org.products.csv.zip', 'r') as zip_ref:
    zip_ref.extractall()

data = pd.read_csv('./fr.openfoodfacts.org.products.csv', sep='\t')

# Data shape analysis

In [ ]:
# Deep copy of the original dataframe
df = data.copy()

In [ ]:
# Quick description of the data
df.shape

In [ ]:
df.describe(include='all')

In [ ]:
df.head()

## Analyse des types de variables

In [ ]:
pd.set_option('display.max_row',18)
graph = df.dtypes.value_counts().plot.pie(title='Analyse de la proportion des types de données',autopct='%1.0f%%',).axis(False)

## Data representation analysis

In [ ]:
# For better understanding of the dataset completion, we try to display the arrangement of the missing values by using a heatmap

# Very heavy to compute
# sns.set()
# ax = plt.axes()
# sns.heatmap(df.isna(), cbar=False, ax = ax)
# ax.set_title('Complétion des données par colonne')
# plt.show()

## Missing values analysis

In [ ]:
# After the visualization, we try to mesure the missing values
pd.set_option('display.max_rows', None)
proportion = 100- (round(df.isna().sum()*100/df.shape[0],2)).sort_values(ascending=True)
print('Completion value rate by columns:')
print(proportion)

Conclusion of the missing values analysis

* Most of the columns are mostly filled with values
* We cannot find if this dataset is a byproduct of multiple other ones as there are not multiple groups of columns with the sate completion rate
* Most of the columns could be worked with, as the less complete one still contains 15% of data (85% of missing values)

In [ ]:
# Usable data
pd.set_option('display.max_rows', 20)
seuil = 70 # Arbitrairy threshold to determine if a column is usable
proportion[proportion > seuil].index

We choose to focus on columns that represent weight per 100g as they seem to have a great completion rate and seem relevant and consitent enough for our analysis

# Data cleaning

## Duplicates and null identifiers

By looking at the dataset, we can see that the `code` columns is not unique as some of them contains duplicate.
We can also predict that this identifier was computed from the column `url` and converted to an int.
e.g. : 
| url (str)                                                | identifier (str) | code (int) |
|----------------------------------------------------------|------------------|------------|
| http://world-fr.openfoodfacts.org/produit/0000000003087/ | 0000000003087    | 3087       |
| http://world-fr.openfoodfacts.org/produit/0000000002867/ | 0000000002867    | 2867(error)|
| http://world-fr.openfoodfacts.org/produit/000002867/     | 000002867        | 2867(error)|

So we decided to use the column `url` as an identifier and to compute any missing value in the column `code` from the column `url` (and vice versa if `url` is missing and not `code`)


In [ ]:

df_nan_code = pd.isna(df['code'])
df_nan_url = pd.isna(df['url'])
df_code_possible = df[df_nan_code != df_nan_url]

print(f"There are {df_code_possible.shape[0]} rows where we can compute either the code or url columns.")

# The purpose of this tool is to redirect the user to the openfoodfacts website, if the url is missing and we cannot reconstruct it from the code, we choose to remove the row.
df = df.drop(df[df_nan_code].index)

# Ultimately, even if the code columns contains duplicates, as we choose to use the url column as Id, this is not useful anymore to clean the code column.
df_duplicated_url = df[df['url'].duplicated()].sort_values('url')
print(f"There are {df_duplicated_url.shape[0]} rows where urls are duplicated. We deleted them.")

df = df.drop(df_duplicated_url.index)

## Outliers selection

In this chapter, we will focus on selection of outliers. To remove them from the dataset.
Mainly, we will sum every 'on 100g' columns to see if it sums above 100g, which will detect one or multiple oultiers. If this is the case, we cannot determine which one is the outlier and we must delete the entire row.

Another problem is that, most of the 'on 100g' columns are included in others. For exemple, it would be an error to sum 'saturated_fat_100g' and 'fat_100g' as 'saturated_fat_100g' is included in 'fat_100g'.
We must try to understand which columns contains the others.
For this exercice, I needed to do a lot of reasearch on biochemistry and nutrition.

### Methodology

We choose to represent columns with a dictionnary because it is a classic object, easy to manipulate and can represent the imbrication of columns.
For exemple, if we have the following dictionnary :
```
a --> a1
  --> a2
b --> b1
  --> b2
c
```
We would have all of this combinations possible
1. `a1 + a2 + b1 + b2 + c`
2. `a1 + a2 + b  + c`
3. `a + b1 + b2  + c `
4. `a + b  + c`

Here, we have a dictionnary with a width of 3 (a, b, c) and a depth of 2 (a1, a2).
It gives us a total of 2^3 = 8 combinations. In reality, we have four different because we "shorten" c.

For a k-width, i-depth dicttionnary, we have i^k combinations. This number can grow so fast and we have so many columns that we decided only to sum 'by levels'.
This means that in our case, we will only have i(=2) combinations, the first one for the first level (a + b + c) and the second one for the second level (a1 + a2 + b1 + b2 + c).
But it also means that if a branch doesn't go down to the deepest level, we have to simulate a deeper level by keeping the same value (as we did for c).

In [ ]:
# Using our reasearch, this is the dictionnary we came with

dict_feature_combinations = {
    'fat_100g':
              {
              'cholesterol_100g':{},
              'saturated-fat_100g':{
                                    'caprylic-acid_100g':{},
                                    'lauric-acid_100g':{},
                                    'myristic-acid_100g':{},
                                    'palmitic-acid_100g':{},
                                    'stearic-acid_100g':{},
                                    'arachidic-acid_100g':{},
                                    'behenic-acid_100g':{},
                                    'lignoceric-acid_100g':{},
                                    'cerotic-acid_100g':{},
                                    'montanic-acid_100g':{},
                                    'melissic-acid_100g':{},
                                    'butyric-acid_100g':{},
                                    'caproic-acid_100g':{},
                                    'capric-acid_100g':{}
                                  },
              'monounsaturated-fat_100g':{
                                          'omega-9-fat_100g':{
                                                            'oleic-acid_100g':{},
                                                            'elaidic-acid_100g':{},
                                                            'gondoic-acid_100g':{},
                                                            'mead-acid_100g':{},
                                                            'erucic-acid_100g':{},
                                                            'nervonic-acid_100g':{}
                                                            }
                                        },
              'polyunsaturated-fat_100g':{
                                        'omega-3-fat_100g':{
                                                          'alpha-linolenic-acid_100g':{},
                                                          'eicosapentaenoic-acid_100g':{},
                                                          'docosahexaenoic-acid_100g':{}
                                                          },
                                        'omega-6-fat_100g':{
                                                          'linoleic-acid_100g':{},
                                                          'arachidonic-acid_100g':{},
                                                          'gamma-linolenic-acid_100g':{},
                                                          'dihomo-gamma-linolenic-acid_100g':{}
                                                          }
                                        },
              'trans-fat_100g':{}
              },
    'sugars_100g':
                {
                    'carbohydrates_100g':{
                                        'sucrose_100g':{},
                                        'glucose_100g':{},
                                        'fructose_100g':{},
                                        'lactose_100g':{},
                                        'maltose_100g':{}
                    }

                },
    'proteins_100g':{
                    'casein_100g':{},
                    }
    }


# This function can either return every string of a dictionnary of return every columns for every levels (depending of the choosen option)
def multi_purpose_function(dico, option):
  # Option 1: get_every_string_of_dict
  # Option 2: get_levels_features

  parent  =''

  level_list = [set(dico.keys())]
  parent_list = [set()]
  string_list = list(dico.keys())
  # Prends toutes les clefs niveau 1
  keys = list(dico.keys())
  # Tant que la boite à clef à explorer n'est pas vide, on continue à explorer
  while (keys != []):
    # Pour chaque clef de la boite à clefs
    for key in keys:
      # On explore

      # On décompose la clef pour avoir le multi level : On a une clef du type lvl1;lvl2;lvl3...
      sublevel = dico
      string_to_print = ''
      level = 0
      for key_level in key.split(';'):
        string_to_print += '--'
        level += 1
        sublevel = sublevel[key_level]
        if level >= len(level_list):
          level_list.append(set())
        if level >= len(parent_list):
          parent_list.append(set())
        

      # On retourne toutes les clefs (sans les dict([]).keys())
      ajout_list = False
      prochaines_clefs = list(sublevel.keys())
      if len(prochaines_clefs) != 0:
        parent = prochaines_clefs[0] 
      for key2 in prochaines_clefs:
        keys.append(f"{key};{key2}")
        string_list.append(key2)
        level_list[level].add(key2)

        if sublevel[key2] != dict([]):
          level_list[level].add(key2)
        else:
          parent_list[level].add(key2)

      # On retire la clef explorée de la boîte à clef
      keys.remove(key)
  
  for i in range(1,level):
    parent_list[i] = parent_list[i].union(parent_list[i-1])
    level_list[i] = level_list[i].union(parent_list[i-1])


  if option == 1:
    return string_list
  if option == 2:
    level_list.pop()
    return level_list

In [ ]:
per_100g_features_list = []

# Get every 'for 100g' column
for index in data.columns:
  if '100' in index and  data[index].dtypes == 'float64': 
    per_100g_features_list.append(index)

# Here we delete the columns that doesn't seem to fit (most of them are just not weights so no possibility to sum them)

not_weight_on_100g_columns = [
                        'energy_100g',
                        'energy-from-fat_100g',
                        'carbon-footprint_100g',
                        'nutrition-score-fr_100g',
                        'nutrition-score-uk_100g', 
                        'glycemic-index_100g',
                        'water-hardness_100g',
                        'ph_100g',
                        'collagen-meat-protein-ratio_100g',
]
for col in not_weight_on_100g_columns :
  per_100g_features_list.remove(col)


# In order not to interfere with the process, every value over 100g is considered as NaN (so will cound as 0 in the sum).
# This is not a problem, as it will be replaced later in the outlier selection process.
for col in per_100g_features_list:
  df.loc[df[col] > 100, [col]] = np.nan

# We select the columns that only are not composed of other columns (the deepest ones)
for feature in multi_purpose_function(dict_feature_combinations,1):
  per_100g_features_list.remove(feature)

# We drop every column that we feel not to fit in the sum for different reasons
col_to_drop = [
          'maltodextrins_100g', # Composed of glucose and fructose. Don't know where to put it.
          'starch_100g', # Disn't find what it is.
          'polyols_100g', # Organic component. Isn't a component of the food.
          'serum-proteins_100g', # Protein coding gene. Don't know where to put it.
          'nucleotides_100g', # Nucleic acid component. Don't know where to put it.
          'beta-carotene_100g', # Precursor of the synthesis of vitamin A. We already study vitamin A.
          'folates_100g', # Same as B9 vitamine
]

for col in col_to_drop:
  per_100g_features_list.remove(col)

# These columns are the deepest ones
per_100g_invar_cols = per_100g_features_list

In [ ]:
# Time to do the sums

set_index_surcharge = set()

for i, colonne_list_variable in enumerate(multi_purpose_function(dict_feature_combinations,2)):
  print(f'Summing the depth level {i+1}')
  colonne_nom = 'somme_100g_n' + str(i)


  # We sum the deepest one with the level 'columns'
  colonne_list = per_100g_invar_cols + list(colonne_list_variable)

  df[colonne_nom] = df[colonne_list].sum(axis=1)
  set_index_surcharge = set_index_surcharge.union(set(df[(df[colonne_nom] > 100) | (df[colonne_nom] < 0)].index))

# When the weight is over 100g for 100g of product, we don't know what columns are wrong so we must delete the entire row

print(f"We delete {len(set_index_surcharge)} rows on {df.shape[0]} which makes {round(len(set_index_surcharge)*100/df.shape[0],2)}%.")
print(f"There will be left {df.shape[0]-len(set_index_surcharge)} rows.")

df.drop(set_index_surcharge)

## Interesting features selection

In [ ]:
# We gather the most interesting features (columns) and set options for them that will serve later

filter_features = pd.DataFrame(data=[
           ['fiber_100g',True, 0,50,0,20],
          #  ['cholesterol_100g',False],
           ['trans-fat_100g',False, 0,1,0,1],
           ['calcium_100g',True,0,2,0,2],
           ['iron_100g',True,0,0.2,0,0.04],
          #  ['energy_100g',True],
           ['proteins_100g',True,0,90,0,30],
          #  ['salt_100g',False],
          #  ['sodium_100g',False],
           ['salt_proc_100g',False,0,100,0,20],
           ['vitamins_count',True,0,11,0,11]
          ],
          columns = ['feature', 'shouldIMaximiseIt', 'min_lim', 'max_lim', 'min_lim_arbitrary', 'max_lim_arbitrary'])




## Artificial features creation

We try to create a new feature counting the number of vitamin in the product. We consider that if a vitamin column is not filled for a product, it doesn't contain it.


In [ ]:
vitamin_columns = []
for index in data.columns:
  if index[0] == 'v' : vitamin_columns.append(index)

# Also adding new vitamins that aren't labeled as 'vitamins'

# Vitamine B5
vitamin_columns.append('pantothenic-acid_100g')
# Vitamine B8
vitamin_columns.append('biotin_100g')

# Create a new column 'vitamins_count' that will count the number of vitamins in the product
vitamins_bool_isna = pd.notna(df[vitamin_columns])
df['vitamins_count'] = vitamins_bool_isna.sum(axis=1)

In [ ]:
# data[vitamin_columns[0:5]][:2]
test = data[vitamin_columns[0:5]][:2].applymap(lambda x: False if ((np.isnan(x)) or (x == 0)) else True)
test['vitamins_count'] = test.sum(axis=1)
test

Depending on the product, either the salt or sodium column is filled, or both.
However, even if it is about the same nutriment, sodium quantity isn't the same as salt quantity for a product.
We can find [here](https://www.wikiwand.com/en/Sodium_chloride) that salt (Sodium Chloride) is composed by 39.34% of sodium and 60.66% of chloride. So `salt = sodium * (100/39.34)`.
In order to use this quantity, we must have a column gathering information about salt using one of the two columns.

In [ ]:
salt_columns = ['salt_100g', 'sodium_100g']

# NaN values in the salt rows
rows_where_salt_na = df['salt_100g'].isna()
# but filled value in the sodium row
rows_where_sodium = df['sodium_100g'].notna()

rows_where_must_calculate_salt = df[rows_where_salt_na & rows_where_sodium].index.tolist()

def _fill_salt_proc_100g_column(x):
  # If there is no salt but sodium, we return the operation, in the otehr case, we simply return the salt value
  return x['sodium_100g'] * (100/39.34) if x.name in rows_where_must_calculate_salt else x['salt_100g']

df['salt_proc_100g'] = df.apply(lambda x: _fill_salt_proc_100g_column(x), axis=1)

## Outliers selection in selected features

The goal is to select outliers in order to process them by other methods later.
Instead of deleting them as soon as we find them, we prefer selecting them for two reasons :
1. Allow to keep NaN values (to process them later)
2. Allow to keep the information about the outliers and visualize them later

For every columns, we determine max and min values that will contain the regular values.

Most of the limits are set thanks to [this report](https://fdc.nal.usda.gov/fdc-app.html#/) of the US department of Agriculture that we will consider as reliable. We also put some more extreme and arbitrary limits to make better visualisations. Later, we will be asked wich one we should keep.


We already set the limits prior in this notebook when defining the interesting features columns.

In [ ]:
df_without_outliers = df.copy()
df_without_outliers_sharp = df.copy()


for index, feature in filter_features.iterrows() :
  feature_name = feature['feature']

  lim_bas_sharp = feature['min_lim_arbitrary']
  lim_haut_sharp = feature['max_lim_arbitrary']
  lim_bas = feature['min_lim']
  lim_haut = feature['max_lim']

  conditions = (df_without_outliers[feature_name] > lim_haut) | (df_without_outliers[feature_name] < lim_bas)
  conditions_sharp = (df_without_outliers_sharp[feature_name] > lim_haut_sharp) | (df_without_outliers_sharp[feature_name] < lim_bas_sharp)

  # Colonnes we will display over the plots
  display_columns = [feature_name,'product_name','brands', 'code']

  df_without_outliers.loc[conditions, feature['feature']] = np.nan
  df_without_outliers_sharp.loc[conditions_sharp, feature['feature']] = np.nan

## Outliers visualization

In [ ]:
# We will visualize our outliers using boxplots

# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(filter_features.shape[0], 3, figsize=(20, 20))
cols = ['Not cleaned', "Cleaned", "Sharp cleaned"]
figure.suptitle('Visualisation of the outliers', fontsize=20)
for ax, col in zip(axis[0], cols):
    ax.set_title(col)
sns.set_theme(style="whitegrid")


for index, column in filter_features.iterrows():
  # Keeping the oultiers
  sns.boxplot(ax=axis[index,0], x=df[column['feature']])

  # Removing the oultiers
  sns.boxplot(ax=axis[index,1], x=df_without_outliers[column['feature']])

  # Removing the oultiers with more fine limits
  sns.boxplot(ax=axis[index,2], x=df_without_outliers_sharp[column['feature']])

figure.tight_layout()
plt.show()

We select the cleaning limits that we prefer.

In [ ]:
# Shoud we use sharp cleaning ?
use_sharp_limits = True

df = df_without_outliers_sharp if use_sharp_limits else df_without_outliers

## Missing values treatment



Interesting features alerady have been selected by using their completion rate, we know that we will be able to work with a lot of data for our prediction.
And as we cleaned the data, we know that we will work with consitent data.

There are several features to fill, so, for each of them (considered as 'target feature'), in order to predict the missing values, we will use the following methods :
1. Determine the 5 more correlated features with the target feature
2. If all of those features are filled, we make a regression to predict the target feature
3. If any of those correlated feature is missing, we chose to use imputation instead


In [ ]:
def choose_most_related_features(df, target, nb, features):
  print('\nChoose_most_related_features')

  if target in features : features.remove(target)

  features_list = features + [target]

  corr = df[features_list].corr()

  corr = corr.drop(target)
  correlated_features = []
  for i in range(nb):
    feature_label = corr[target].abs().idxmax()
    print(f"-- Feature selected n°{i+1} : {feature_label} with corr {round(corr[target][feature_label],3)*100}%")
    correlated_features.append(feature_label)
    corr = corr.drop(feature_label)
  return correlated_features

In order to have a precise imputation method, we will use a sample of the dataset corresponding of the same 'group' of the imputed row.
e.g. : If we must predict 'fiber_100g' columns for a product in the group 'fruit juice', we will use the sample of the dataset corresponding to the group 'fruit juice' because using the 'meat' group would be incoherent.

In [ ]:
from sklearn.impute import SimpleImputer

def make_imputation(df, target, method):
  print('\nImputation')
  df['pnns_groups_2'] = df['pnns_groups_2'].apply(lambda x : 'nan' if pd.isna(x) else x) # Do this because if not, it is impossible select the nan group

  for group2 in df['pnns_groups_2'].unique():
    sub_df = df[df['pnns_groups_2'] == str(group2)][target]
    print(f"------ {group2} --> {sub_df.shape[0]} row and {sub_df.isna().values.sum()} imputations found !")
    sub_df[sub_df.isna()]['target_imputed'] = 1
    imputer = SimpleImputer(missing_values=np.nan, strategy=method)
    imp_ser = imputer.fit_transform(sub_df.values.reshape(-1, 1))
    df.loc[df['pnns_groups_2'] == group2, target] = imp_ser

  return df

In [ ]:
from sklearn import linear_model

def make_regression(df, features, target):
  print('\nRegression')
  # We split the dataset into two groups, the one where the target is filled (to train the regression) and the other where the target is missing (to make the prediction).
  columns_used = features + [target]
  train_df = df[columns_used]
  train_df = train_df.dropna()
  
  train_features = train_df[features]
  train_target = train_df[target]


  predict_df = df[df[target].isna()]

  predict_df = predict_df[features].dropna()
  if (predict_df.shape[0] == 0):
    print('-- Not enough valid features to make any prediction. At least one feature in each prediction row is missing. Will do it by imputation.')
  else:
    print(f"-- {predict_df.shape[0]} rows eligible to prediction")
    predict_features = predict_df[features]

    X = train_features
    y = train_target
    regr = linear_model.LinearRegression()
    print('---- fitting...')
    regr.fit(X, y)
    print('---- fitted')
    predict_df[target] = regr.predict(predict_features)
    df.loc[predict_df.index,target] = predict_df
  return df

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

nb_correlated_features = 5
interesting_features = proportion[proportion > seuil].index.tolist()


for index, target in enumerate(filter_features['feature'].tolist()):
  # On rajoute une colonne flag pour si la target a été imputed ou non.
  # Ca sert pour amélriorer la regression.
  # Ce flag est reset pour chaque nouvelle target
  df['target_imputed'] = 0
  
  print(f"\n\n_________________________________________________________________\nFilter n°{index+1} : {target}")

  most_related_features = choose_most_related_features(df, target , nb_correlated_features, interesting_features)
  print(f"\n{df[target].isna().sum()} targets left to predict")

  regression_df = make_regression(df, most_related_features, target)
  print(f"\n{regression_df[target].isna().sum()} targets left to impute")

  imputed_df = make_imputation(regression_df, target, 'mean') # Faire une moyenne en prenant la même catégorie de produits

  assert imputed_df[target].isna().sum() == 0, f"imputation failed, there are still missing values in {target}"

  df.loc[:,target] = imputed_df[target]

# Exploratory analysis of the cleaned dataset

In [ ]:
corr = df[interesting_features].corr()


# To show heatmap
fig, axs = plt.subplots(1,1,figsize=(5,5))
sns.heatmap(corr)
plt.show()

An interesting fact is that ingredient_that_may_be_from_palm oil is greatly correlated with additives.
This is not a correlation we exploit later but deserve to be noticed.

In [ ]:
# Analysis of the data repartition on the interesting features columns after cleaning

# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(filter_features.shape[0], 2, figsize=(20, 20))
cols = ['Histogramme','Boite à moustache']
for ax, col in zip(axis[0], cols):
    ax.set_title(col)
sns.set_theme(style="whitegrid")


for index, column in filter_features.iterrows():
  # histograme
  sns.histplot(ax=axis[index,0], x=df[column['feature']])

  # Boxplot
  sns.boxplot(ax=axis[index,1], x=df[column['feature']])

figure.tight_layout()
plt.show()

We can notice that the linear regression has created some outliers (for example with negative values of fiber).

## Principal component analysis

In [ ]:
# Functions definition

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram

# Taken from here https://github.com/stenier-oc/realisez-une-analyse-de-donnees-exploratoire/blob/master/functions.py
def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    for d1, d2 in axis_ranks: # On affiche les 3 premiers plans factoriels, donc les 6 premières composantes
        if d2 < n_comp:

            # initialisation de la figure
            fig, ax = plt.subplots(figsize=(7,6))

            # détermination des limites du graphique
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # affichage des flèches
            # s'il y a plus de 30 flèches, on n'affiche pas le triangle à leur extrémité
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (voir la doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # affichage des noms des variables  
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # affichage du cercle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # définition des limites du graphique
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # affichage des lignes horizontales et verticales
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Cercle des corrélations (F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)
        
def display_factorial_planes(X_projected, n_comp, pca, axis_ranks, labels=None, alpha=1, illustrative_var=None):
    for d1,d2 in axis_ranks:
        if d2 < n_comp:
 
            # initialisation de la figure       
            fig = plt.figure(figsize=(7,6))
        
            # affichage des points
            if illustrative_var is None:
                plt.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha)
            else:
                illustrative_var = np.array(illustrative_var)
                for value in np.unique(illustrative_var):
                    selected = np.where(illustrative_var == value)
                    plt.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value)
                plt.legend()

            # affichage des labels des points
            if labels is not None:
                for i,(x,y) in enumerate(X_projected[:,[d1,d2]]):
                    plt.text(x, y, labels[i],
                              fontsize='14', ha='center',va='center') 
                
            # détermination des limites du graphique
            boundary = np.max(np.abs(X_projected[:, [d1,d2]])) * 1.1
            plt.xlim([-boundary,boundary])
            plt.ylim([-boundary,boundary])
        
            # affichage des lignes horizontales et verticales
            plt.plot([-100, 100], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-100, 100], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué
            plt.xlabel('F{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('F{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Projection des individus (sur F{} et F{})".format(d1+1, d2+1))
            plt.show(block=False)

def display_scree_plot(pca):
    scree = pca.explained_variance_ratio_*100
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage d'inertie")
    plt.title("Eboulis des valeurs propres")
    plt.show(block=False)

def plot_dendrogram(Z, names):
    plt.figure(figsize=(10,25))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('distance')
    dendrogram(
        Z,
        labels = names,
        orientation = "left",
    )
    plt.show()

In [ ]:
# PCA realisation

from sklearn import decomposition
from sklearn import preprocessing

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

floatInterrestingFeatures = [col for col in df[interesting_features].columns if df[col].dtypes == 'float64'] 
data = df[filter_features['feature']]

# Selection of the number of PCA components
n_comp = min(data.shape[0], data.shape[1]-1)

# Centering and Reduction
std_scale = preprocessing.StandardScaler().fit(data.values)
X_scaled = std_scale.transform(data.values)


# PCA calculations
pca = decomposition.PCA(n_components=n_comp)


pca.fit(X_scaled)

# Cumulated inertia
display_scree_plot(pca)

# Correlation circles
pcs = pca.components_
display_circles(pcs, n_comp, pca, [(0,1),(2,3),(4,5)], labels = np.array(data.columns))

# # Data projection (doesn't work properly)
# X_projected = pca.transform(X_scaled)
# display_factorial_planes(X_projected, n_comp, pca, [(0,1),(2,3),(4,5)], labels = np.array(data.index))

# plt.show()

# ANOVA

## 1-variable ANOVA : pnns_groups_2

In [ ]:
# Target repartition analisys depending of the group

sns.set(font_scale = 2)
figure, axis = plt.subplots(filter_features.shape[0], 2, figsize=(40,100))


cols = ['Histogramme','Boite à moustache']
axis[0, 0].set_title(cols[0],fontsize=20)
axis[0, 1].set_title(cols[1],fontsize=20)

sns.set_theme(style="whitegrid")


for index, feature in filter_features.iterrows():

  
  axHist = axis[index, 0]
  axBoxP = axis[index, 1]

  # On rajoute le titre pour chaque feature
  axHist.set_title(feature['feature'],fontsize=40)
  axBoxP.set_title(feature['feature'],fontsize=40)

  axBoxP.tick_params(axis='x', rotation=90)


  # histograme
  sns.kdeplot(ax=axis[index,0], data=df,  x = feature['feature'], hue = 'pnns_groups_2')
  plt.setp(axis[index,0].get_legend().get_texts(), fontsize='12') # for legend text
  plt.setp(axis[index,0].get_legend().get_title(), fontsize='22') # for legend title

  # Boxplot
  sns.boxplot(ax=axis[index,1], data=df, x='pnns_groups_2', y = feature['feature'])



figure.tight_layout()
plt.show()

## Hypothese redaction

H0 : I think that the variable 'pnns_groups_2' has no influence on the salt rate.

In [ ]:
%pip install statsmodels
import statsmodels.formula.api as smf
import statsmodels.api as sm


anova_group_salt = smf.ols('salt_proc_100g~pnns_groups_2', data=df).fit()
print(anova_group_salt.summary())

if sm.stats.anova_lm(anova_group_salt, typ=2)['PR(>F)'][f"pnns_groups_2"] >= 0.05 :
  print('\n\n\nBy fisher test, the group has no significant influence on salt rate. H0 is rejected.')
else:
  print('\n\n\nBy fisher test, the group has a significant influence on salt rate. H0 is accepted.')

## 1-variable ANOVA : Seeking for a qualitative variable uncorrelated with the salt rate



In [ ]:
for feature in df.columns :
  another_df = df.copy()
  if ((len(df[feature].unique()) < 500) & (df[feature].dtypes == 'object') & (len(df[feature].unique()) > 1)):
    another_df.loc[df[feature].isna(), feature] = 'nan'
    anova_categorie_salt = smf.ols(f'salt_proc_100g~{feature}', data=another_df).fit()
    if sm.stats.anova_lm(anova_categorie_salt, typ=2)['PR(>F)'][f"{feature}"] >= 0.05 :
      print(f"{feature} has no significant influence on salt rate.")
      print(anova_categorie_salt.summary())
      df = another_df.copy() # We keep every modification we made on the 'not influent feature' as we will reuse it later
      break 

## 2-variable ANOVA

We know that the variable 'pnns_groups_2'has a influence on the salt rate, which isn't the case for the variable 'ingredients_from_palm_oil_tags'.

So we will do another ANOVA with the two variables in order to know if 'pnns_groups_2' has an influence on the salt rate for each 'pnns_groups_2' value.

In [ ]:
cat1 = 'pnns_groups_2'
cat2 = 'ingredients_from_palm_oil_tags'
target = 'salt_proc_100g'

formula = f'{target}~{cat1}*{cat2}'
anova_interactions_salt = smf.ols(formula, data=df).fit()
print(anova_interactions_salt.summary())
anova_lm = sm.stats.anova_lm(anova_interactions_salt)
print(anova_lm)

if sm.stats.anova_lm(anova_interactions_salt, typ=2)['PR(>F)'][f"{cat1}:{cat2}"] >= 0.05 :
  print(f"\n\n\nInteraction between {cat1} and {cat2} hasn't any noticeable influence on the salt rate using Fisther test.")
else:
  print(f"\n\n\nInteraction between {cat1} and {cat2} has a noticeable influence on the salt rate using Fisther test.")

# Dynamic filtering

Le but est que l'utilisateur puisse choisir le (ou les critères) qui lui conviennent le plus et rentre un nom de produit. La base de donnée lui permettra de trouver ce qui correspond le mieux à sa demande et lui fournira aussi un histogramme regroupant tous les produits concurrents et leurs avantages.

Critères :
* fiber_100g
* additives_fr
* additives_tags
* cholesterol_100g
* trans-fat_100g
* calcium_100g
* vitamin-c_100g
* iron_100g
* vitamin-a_100g
* energy_100g
* proteins_100g
* salt_100g
* sodium_100g

## Entrées Utilisateur

Parameters are
1. Which filters
2. What Product
3. How many other products to compare
4. Which location (later)

In [ ]:
def ask_user_parameters():
  user_product_name = input('Veuillez entrer un nom de produit :\n')

  print(f"These are the differents filters you can select.\nPlease type their number for adding them to the filter list or . for stopping the selection.")
  features_list = {str(index+1):str(feature) for index,feature in enumerate(filter_features['feature'].tolist())}
  for index, feature in features_list.items():
      print(f"{index}.  {feature}")

  fin_de_selection = False
  filter_set = set()
  while fin_de_selection == False:
    user_filter = input('Votre filtre ?\n')
    if (user_filter in features_list):
      filter_set.add(features_list[user_filter])
      print(f"Voici votre liste de filtres actuels :\n{filter_set}")
    elif (user_filter == '.'):
      fin_de_selection = True
    else :
      print('Sélection non reconnue. Réessayez s\'il vous plait.')

  nb_res = 'a'
  while nb_res.isdigit() == False:
    nb_res = input('En plus du meilleur résultat, de combien autre produit voulez-vous voir le comparatif ?\n')
  nb_res = int(nb_res)
  
  return list(filter_set), user_product_name, nb_res

In [ ]:
def pick_random_parameters(df):

  nb_filter = np.random.randint(1,6) # De 1 à 5
  filters = np.random.choice(filter_features['feature'].tolist(), nb_filter, replace=False)
  top_nb = np.random.randint(3, 11) # De 3 à 10

  print(f"Filters are {filters}")

  # Pick one or more product (with enough products registered)
  sub_df_value_counts = df['product_name'].value_counts() > 50
  sub_df_value_counts = sub_df_value_counts[sub_df_value_counts.values == True].index

  product_name = np.random.choice(sub_df_value_counts, 1)[0]
  print(f"Product name is {product_name}")

  return filters, product_name, top_nb

In [ ]:
entree_reconnue = False

while entree_reconnue == False:
  # user_input = input('Voulez vous entrer les données utilisateur ? (Y/N)')
  user_input = 'N'
  if (user_input == 'Y'):
    entree_reconnue = True
    filters, product_name, nb_res = ask_user_parameters()
  elif (user_input == 'N'):
    entree_reconnue = True
    filters, product_name, nb_res = pick_random_parameters(df)
  else :
    print('Entrée non reconnue')

## Ranking

Le ranking se fait comme suit :
1. Pour chaque feature, on effectue un ranking des individus (voir [cette méthode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rank.html)). __Attention, certaines features doivent être traitées de sorte à maximiser ou minimiser les quantités (ex : protéines et sel).__
2. On effectue la somme des ranking de ces features par individu
3. Enfin, on rank de nouveau cette somme afin d'avoir un classement. Cette fois-ci, on utilise l'option `method='dense'` afin d'avoir un score de type ordinal.
4. Après un trie du meilleur score au moins bon, on peut séléctionner le premier individu (gagnant ainsi que les suivants).

In [ ]:
selected_df = df

# On séléctionne les produits par le nom
product_list = selected_df[selected_df['product_name'].str.contains(product_name, case=False, na=False)]

list_score_col_label = set()
for index, feature in enumerate(filters):
  should_maximise = filter_features.loc[filter_features['feature'] == feature,('shouldIMaximiseIt')].iloc[0]
  print(f"feature is {feature} and should I maximise it ? {should_maximise}")
  product_list[feature + '_rank'] = product_list[feature].rank(ascending=should_maximise)
  list_score_col_label.add(feature + '_rank')

product_list['sum_scores_rank'] = product_list[list_score_col_label].sum(axis=1)
product_list['multiple_rank'] = product_list['sum_scores_rank'].rank()
best = product_list.nlargest(nb_res+1, 'multiple_rank')
product_list_size = product_list.shape[0]
best

## Affichage des meilleurs produits

In [ ]:
plt.figure(figsize=(10,10))


if len(list_score_col_label) > 1:
  new_perc_cols = set()
  list_score_col_label.add('multiple_rank')

  # Pour que ce soit plus lisible dans le graphique, on va noter chaque rank en pourcentage (ex : 6ème sur 10 --> (6/10) * 100 = au dessus de 60% de l'échantillon)
  for feature_rank in list_score_col_label:
    col_name = feature_rank + '_perc'
    best[col_name] = best[feature_rank]*100/product_list_size
    new_perc_cols.add(col_name)
  
else : # Si il n'y a qu'une seule colonne, on préfere afficher la colonne en question plutôt qu'un ranking par rapport aux autres
  new_perc_cols = filters

# On rajoutera aussi aux brand names le nom du produit
best['new_name'] = best['brands'] + '\n' + best['product_name']




# Avec matplotlib
# best.plot(x='new_name', y=new_perc_cols, kind='bar')
# plt.show()



# Avec sns
best_sns = best.melt(id_vars="new_name")

best_sns = best_sns.drop(best_sns[~best_sns['variable'].isin(new_perc_cols)].index).sort_values('value', ascending=False)

fig, axs = plt.subplots(1,1,figsize=(len(new_perc_cols)*5,5))
sns.barplot(x='new_name', y='value', hue='variable', data=best_sns, ax=axs)
axs.tick_params(axis='x', rotation=90, labelsize=10)
axs.set_title(f"Ranking of {product_name} by {filters}",fontsize=20)
axs.set_xlabel("Product name",fontsize=20)
axs.set_ylabel("Ranking (in %)",fontsize=20)
axs.set_ylim(bottom = 0)
axs.tick_params()
plt.show()

In [ ]:
# https://stackoverflow.com/questions/10371200/get-image-from-website

%pip install requests
import requests

import os

%pip install tqdm
from tqdm import tqdm

%pip install bs4
from bs4 import BeautifulSoup as bs

%pip instamm urllib
from urllib.parse import urljoin, urlparse

import matplotlib.image as mpimg
%pylab inline


def get_picture_from_url(url):
  def get_all_images(url):
      """
      Returns all image URLs on a single `url`
      """
      soup = bs(requests.get(url).content, "html.parser")

      urls = []
      for img in tqdm(soup.find_all("img"), "Extracting images"):
          img_url = img.attrs.get("src")
          if not img_url:
              # if img does not contain src attribute, just skip
              continue
          # make the URL absolute by joining domain with the URL that is just extracted
          img_url = urljoin(url, img_url)
          try:
              pos = img_url.index("?")
              img_url = img_url[:pos]
          except ValueError:
              pass
          # finally, if the url is valid
          if bool(urlparse(img_url).netloc) and bool((urlparse(img_url).scheme)):
              urls.append(img_url)
      return urls

  filename = ''
  file_name_start = 'front_en.'
  file_name_end = '.full.jpg'
  imgs = get_all_images(url)
  for img in imgs:
    if ((file_name_start in img) and (img.endswith(file_name_end))):
      # download the body of response by chunk, not immediately
      response = requests.get(img, stream=True)
      # get the total file size
      file_size = int(response.headers.get("Content-Length", 0))
      # get the file name
      filename = os.path.join('./', img.split("/")[-1])
      # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
      progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
      with open(filename, "wb") as f:
          for data in progress.iterable:
              # write data read to the file
              f.write(data)
  if filename == '':
    print('\nAucune image trouvée')
  else:
    print(f"\nfile name = {filename}")
    pictures.append(filename)

pictures = []
for url in best['url']:
  get_picture_from_url(url)

for picture in pictures:
  import matplotlib.pyplot as plt
  img = mpimg.imread(picture)
  imgplot = plt.imshow(img)
  plt.show()